### Import Section

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

###  Data setup

In [3]:
! ls data/wiki_valid/val_test

wiki_valid_100K.txt


In [4]:
! ls data/wiki_train/train_test/

wiki_train_1000K.txt


In [5]:
#! rm data/wiki_train/train_test/wiki_train_1K.txt
#! head -n 1000000 data/wiki_train/train_90_10/wiki_train_90pct.txt > data/wiki_train/train_test/wiki_train_1000K.txt

#! rm data/wiki_valid/val_test/wiki_test_1K.txt
#! head -n 100000 data/wiki_valid/valid_90_10/wiki_valid_10pct.txt > data/wiki_valid/val_test/wiki_valid_100K.txt


rm: cannot remove 'data/wiki_train/train_test/wiki_train_1K.txt': No such file or directory
rm: cannot remove 'data/wiki_valid/val_test/wiki_test_1K.txt': No such file or directory


In [6]:
PATH='data/'

TRN_PATH = 'wiki_train/train_test/'
VAL_PATH = 'wiki_valid/val_test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

aclImdb/                    kk_train/  models/  wiki_en.txt  wiki_valid/
cleaned-extra-kk-jokes.txt  kk_valid/  tmp/     wiki_train/


In [7]:
trn_files = !ls -U {TRN}
trn_files

['wiki_train_1000K.txt']

In [8]:
review = !head {TRN}{trn_files[0]}
review[0]

'<Article>'

In [9]:
!find {TRN} -name '*.txt' | xargs cat | wc -w

288749349


In [10]:
!find {VAL} -name '*.txt' | xargs cat | wc -w

29881910


### Creating the model

In [11]:
spacy_tok = spacy.load('en')

In [12]:
TEXT = data.Field(lower=True, tokenize='spacy')

In [13]:
bs=64; bptt=70

In [14]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=1)

In [15]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT_wiki.pkl','wb'))

batches; unique tokens in the vocab; tokens in the training set; sentences

In [17]:
print(f'no. of batches: {len(md.trn_dl)}\nunique tokens: {md.nt}\ntokens in training \
set:{len(md.trn_ds)}\nsentences:{len(md.trn_ds[0].text)}')

no. of batches: 64685
unique tokens: 2008870
tokens in training set:1
sentences:289797265


### Training

In [18]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [19]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [20]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [21]:
import time

In [ ]:
t0=time()
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

In [ ]:
learner.save_encoder('adam1_enc_wiki1M')

In [ ]:
t1=time()-t0
print('time to train: {:.2f} sec'.format(t1)

In [ ]:
learner.load_encoder('adam1_enc_wiki1M')

In [ ]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

In [ ]:
learner.save_encoder('adam3_10_enc_wiki1M')

In [ ]:
learner.load_encoder('adam3_10_enc_wiki1M')

In [ ]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20)

In [ ]:
learner.save_encoder('adam3_20_enc_wiki1M')

In [ ]:
learner.load_encoder('adam3_20_enc_wiki1M')

In [ ]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT_wiki1M_2.pkl','wb'))

### Testing

In [ ]:
m=learner.model
ss=""". So, it wasn't quite was I was expecting, but I really liked it anyway! The best"""
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

In [ ]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [ ]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

In [ ]:
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

# Archive

In [ ]:
learner = model_data.get_model(optimization_function, 
                               embedding_matrix_vector_size,
                               hidden_activations_per_layer,
                               number_of_layers,
                               dropouti=0.05, 
                               dropout=0.05, 
                               wdrop=0.1, 
                               dropoute=0.02, 
                               dropouth=0.05)

In [ ]:
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)

In [ ]:
# This clips the learning rate
learner.clip=0.2

In [ ]:
learner.fit(3e-4, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

In [ ]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

In [ ]:
model = learner.model

In [ ]:
ss = "m"

In [ ]:
s = [TEXT.tokenize(ss)]
t = TEXT.numericalize(s)
' '.join(s[0])

In [ ]:
model[0].bs = 1
model.eval()
model.reset()
res, *_ = model(t)
model[0].bs = bs

In [ ]:
next_words = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(next_words)]

In [ ]:
print(ss,"\n")
for i in range(300):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = model(n[0].unsqueeze(0))
print('...')